# 🎲 PyKwant Tutorial: Monte Carlo Simulations

This notebook introduces `pykwant.simulations`, a module for pricing complex derivatives using stochastic simulation.

While Black-Scholes works for standard European options, it fails for **Path-Dependent Options** where the payoff depends on the entire history of prices, not just the final price.

We will compare pricing for:
1. **European Call** (Benchmark against Black-Scholes).
2. **Asian Call** (Payoff based on Average Price).
3. **Lookback Call** (Payoff based on Minimum Price).

## 1. Setup Market Environment

We define a standard environment: Spot 100, Vol 20%, Risk-Free 5%, Time 1 Year.

In [1]:
import math
from datetime import date

from pykwant import dates, equity, instruments, simulations

# Market Parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
sigma = 0.20

# Simulation Parameters
N_STEPS = 252  # Daily steps for 1 year
N_PATHS = 10_000  # Number of scenarios
SEED = 42  # Reproducibility

print(f"Simulating {N_PATHS} paths with {N_STEPS} steps each...")

Simulating 10000 paths with 252 steps each...


## 2. Generate Paths (Geometric Brownian Motion)

We use the `generate_paths_gbm` function. This creates thousands of possible future price scenarios.

In [2]:
paths = simulations.generate_paths_gbm(
    s0=S0, drift=r, volatility=sigma, time_horizon=T, steps=N_STEPS, num_paths=N_PATHS, seed=SEED
)

# Let's inspect the first path (first 5 prices)
print("Sample Path 0 (First 5 days):", [round(p, 2) for p in paths[0][:5]])

Sample Path 0 (First 5 days): [100.0, 99.83, 99.63, 99.5, 100.39]


## 3. Benchmarking: European Call

First, we price a standard European Call to verify our Monte Carlo engine works. It should match the Black-Scholes price closely.

In [3]:
# 1. Analytical Price (Black-Scholes)
val_date = date(2025, 1, 1)
expiry = date(2026, 1, 1)


def flat_curve(d):
    return math.exp(-r * dates.act_365(val_date, d))


opt = instruments.EuropeanOption("TEST", instruments.Money(K), expiry, "call")

bs_price = equity.black_scholes_price(opt, S0, sigma, flat_curve, val_date)

# 2. Monte Carlo Price
df = math.exp(-r * T)
payoff_eu = simulations.payoff_european_call(K)

mc_price_eu = simulations.monte_carlo_price(paths, payoff_eu, df)

print(f"Black-Scholes Price: {bs_price:.4f}")
print(f"Monte Carlo Price:   {mc_price_eu:.4f}")
print(f"Difference:          {mc_price_eu - bs_price:.4f}")

Black-Scholes Price: 10.4506
Monte Carlo Price:   10.5042
Difference:          0.0536


## 4. Asian Option Pricing

An **Asian Option** payoff depends on the **Average Price** over the option's life.
$$ Payoff = \max( \text{Average}(S) - K, 0 ) $$

Asian options are typically cheaper than European options because the averaging process smoothens out volatility.

In [4]:
# Use the factory for Asian Arithmetic Payoff
payoff_asian = simulations.payoff_asian_arithmetic_call(K)

mc_price_asian = simulations.monte_carlo_price(paths, payoff_asian, df)

print(f"Asian Call Price:    {mc_price_asian:.4f}")
print(f"Discount vs European: {(1 - mc_price_asian / mc_price_eu):.1%} Cheaper")

Asian Call Price:    5.7149
Discount vs European: 45.6% Cheaper


## 5. Lookback Option Pricing

A **Fixed Strike Lookback Call** pays off the difference between the **Maximum Price** achieved and the Strike.
$$ Payoff = \max( \max(S) - K, 0 ) $$

This option allows you to "buy at the strike and sell at the peak". It is very expensive.

In [5]:
# Custom Payoff Function (Closure)
def payoff_lookback_call(path):
    max_price = max(path)
    return max(max_price - K, 0.0)


mc_price_lookback = simulations.monte_carlo_price(paths, payoff_lookback_call, df)

print(f"Lookback Call Price: {mc_price_lookback:.4f}")
print(f"Premium vs European: {(mc_price_lookback / mc_price_eu - 1):.1%} More Expensive")

Lookback Call Price: 18.2923
Premium vs European: 74.1% More Expensive


## 6. Convergence Analysis

Monte Carlo accuracy improves with $\sqrt{N}$. Let's see how the price stabilizes as we add more paths.

In [6]:
counts = [100, 1000, 5000, 10000]

print(f"{'Paths':<10} | {'Est. Price':>12}")
print("-" * 25)

for n in counts:
    subset_paths = paths[:n]
    p = simulations.monte_carlo_price(subset_paths, payoff_eu, df)
    print(f"{n:<10} | {p:>12.4f}")

Paths      |   Est. Price
-------------------------
100        |      11.2606
1000       |      10.8735
5000       |      10.4358
10000      |      10.5042
